In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from tqdm import tqdm
import sys
sys.path.append("..")

import config

In [29]:
domain = "10YFR-RTE------C"

# DOMAINS
# be: 10YBE----------2
# de/lu: 10Y1001A1001A82H
# de/at/lu: 10Y1001A1001A63L
# nl: 10YNL----------L
# fr: 10YFR-RTE------C

dates = range(2015, 2023)
renewablesDAH_df = pd.DataFrame([], columns = ["datetime", "solar", "wind_onshore", "wind_offshore"])

In [30]:
def get_response(year, BZ, typ):
    
    """
    This function takes in a year and a bidding zone.
    It calls the API, and returns the API response.
    This can be fed to the data format function.
    """
    
    startdate = str(year) + "01010000"
    enddate = str(year) + "12312300"
    domain = BZ
    
    try:
        response = requests.get("https://web-api.tp.entsoe.eu/api",
                        params = {"securityToken": config.security_token,
                                "documentType": "A69",
                                "processType": "A01",
                                "psrType": typ,
                                "in_Domain": domain,
                                "periodStart": startdate,
                                "periodEnd": enddate})
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    return response

In [31]:
def parse_data(resp):
    """
    This function takes in a response from the API.
    It returns a pandas dataframe with variables start and price. 
    Price is a number, and start is a datetime variable.
    The index starts at 0 and goes until the end.
    This function is to be executed per year, per bidding zone.
    Resulting dataframes can then later be combined.
    """
    response = resp

    root = ET.fromstring(response.content)
    timeseries = root.findall("./{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}TimeSeries")

    df_cols = ["start", "position", "quantity"]
    df_rows = []

    for ts in timeseries:
        data = ts[7]
        for day in data[2:]:
            df_rows.append({
                "start": data[0][0].text,
                "position": day[0].text,
                "quantity": day[1].text
            })

    combined_df = pd.DataFrame(df_rows, columns = df_cols)

    #print(f"Resolution {data[1].text}")

    combined_df["start"] = combined_df["start"].astype("datetime64")
    combined_df["position"] = pd.to_timedelta(combined_df["position"].astype("int"), unit="h")
    out_df = pd.DataFrame([], columns = ["datetime", "generation"])
    out_df["datetime"] = combined_df["start"] + combined_df["position"] - pd.to_timedelta(1, unit="h")
    out_df["generation"] = combined_df["quantity"]

    return out_df

In [32]:
# psr type is a queried production type
# B18 wind onshore
# B19 wind ofshore
# b16 solar
# b14 nuclear

In [33]:
# solar
# B16
solarDAH_df = pd.DataFrame([], columns=["datetime", "solar"])
for year in tqdm(dates):
    resp = get_response(year, domain, "B16")
    out_df = parse_data(resp)
    out_df.rename(columns={"generation": "solar"}, inplace=True)
    solarDAH_df = pd.concat([solarDAH_df, out_df], axis=0, ignore_index=True)

100%|█████████████████████████████████████████████| 8/8 [00:25<00:00,  3.15s/it]


In [34]:
# wind onshore
# B19
windonshoreDAH_df = pd.DataFrame([], columns=["datetime", "wind_onshore"])
for year in tqdm(dates):
    resp = get_response(year, domain, "B19")
    out_df = parse_data(resp)
    out_df.rename(columns={"generation": "wind_onshore"}, inplace=True)
    windonshoreDAH_df = pd.concat([windonshoreDAH_df, out_df], axis=0, ignore_index=True)

100%|█████████████████████████████████████████████| 8/8 [00:25<00:00,  3.25s/it]


In [35]:
# wind offshore
# B18
windoffshoreDAH_df = pd.DataFrame([], columns=["datetime", "wind_offshore"])
for year in tqdm(dates):
    resp = get_response(year, domain, "B18")
    out_df = parse_data(resp)
    out_df.rename(columns={"generation": "wind_offshore"}, inplace=True)
    windoffshoreDAH_df = pd.concat([windoffshoreDAH_df, out_df], axis=0, ignore_index=True)

100%|█████████████████████████████████████████████| 8/8 [00:04<00:00,  1.64it/s]


In [36]:
# this one is only for France
renewablesDAH_df = solarDAH_df.merge(windonshoreDAH_df,on='datetime')

In [27]:
renewablesDAH_df = solarDAH_df.merge(windonshoreDAH_df,on='datetime').merge(windoffshoreDAH_df,on='datetime')

In [37]:
renewablesDAH_df

,datetime,solar,wind_onshore
0,2014-12-31 23:00:00,0,1440
1,2015-01-01 00:00:00,0,1457
2,2015-01-01 01:00:00,0,1579
3,2015-01-01 02:00:00,0,1703
4,2015-01-01 03:00:00,0,1829
...,...,...,...
69365,2022-12-31 18:00:00,0,15227
69366,2022-12-31 19:00:00,0,15020
69367,2022-12-31 20:00:00,0,14813
69368,2022-12-31 21:00:00,0,14605


In [38]:
renewablesDAH_df.to_csv("../data/renewablesDAH_FR.csv", index=False)